In [1]:
# Optional helpers: make JSON-like names safe if accidentally typed unquoted in a cell
try:
    null
except NameError:
    null = None
try:
    true
except NameError:
    true = True
try:
    false
except NameError:
    false = False



In [ ]:
from pathlib import Path
import uuid
import re

ALLOWED_EXTENSIONS = {
    ".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".tif", ".webp", ".heic", ".heif"
}


def prompt_for_directory() -> Path:
    folder_input = input("Enter the path to the folder containing images: ").strip().strip('"').strip("'")
    folder_path = Path(folder_input).expanduser().resolve()
    if not folder_path.exists():
        raise FileNotFoundError(f"Folder does not exist: {folder_path}")
    if not folder_path.is_dir():
        raise NotADirectoryError(f"Not a directory: {folder_path}")
    return folder_path


def prompt_for_digit_label(default: str = "1") -> str:
    raw = input(f"Enter the digit label (0-9) for names (default '{default}'): ").strip()
    if raw.lower() in {"null", "none"}:
        raw = ""
    value = raw or default
    value = re.sub(r"[^0-9]", "", value)  # keep digits only
    return value or default


def prompt_for_start_index(default: int = 0) -> int:
    raw = input(f"Enter start index (default {default}): ").strip()
    if raw.lower() in {"null", "none", ""}:
        return default
    try:
        val = int(raw)
        return max(0, val)
    except ValueError:
        return default


def find_image_files(folder: Path):
    image_paths = [p for p in folder.iterdir() if p.is_file() and p.suffix.lower() in ALLOWED_EXTENSIONS]
    image_paths.sort(key=lambda p: p.name.lower())
    return image_paths


def rename_images_digit_style(folder: Path, digit_label: str, start_index: int = 0):
    image_files = find_image_files(folder)
    if not image_files:
        print("No image files found to rename.")
        return

    print(f"Found {len(image_files)} image(s). Renaming to DIGIT_{digit_label}_{start_index}..DIGIT_{digit_label}_{start_index + len(image_files) - 1} while preserving extensions...")

    temp_files = []
    for original_path in image_files:
        temp_name = f"{original_path.stem}__TMP_RENAME__{uuid.uuid4().hex}{original_path.suffix}"
        temp_path = original_path.with_name(temp_name)
        original_path.rename(temp_path)
        temp_files.append(temp_path)

    renamed_paths = []
    try:
        for idx, temp_path in enumerate(temp_files, start=start_index):
            final_name = f"DIGIT_{digit_label}_{idx}{temp_path.suffix}"
            final_path = folder / final_name
            if final_path.exists():
                raise FileExistsError(f"Final target already exists: {final_path}")
            temp_path.rename(final_path)
            renamed_paths.append(final_path)
    except Exception as exc:
        print(f"Error during renaming: {exc}. Attempting best-effort rollback...")
        for path in folder.iterdir():
            if path.is_file() and "__TMP_RENAME__" in path.name:
                cleaned_name = path.name.replace("__TMP_RENAME__", "")
                try:
                    path.rename(folder / cleaned_name)
                except Exception:
                    pass
        raise

    print(f"Renamed {len(renamed_paths)} image(s) in: {folder}")
    for preview in renamed_paths[:10]:
        print(f"- {preview.name}")
    if len(renamed_paths) > 10:
        print("...")


if __name__ == "__main__":
    target = prompt_for_directory()
    digit = prompt_for_digit_label("0")
    start = prompt_for_start_index(1)
    rename_images_digit_style(target, digit, start)



Found 100 image(s). Renaming to DIGIT_0_1..DIGIT_0_100 while preserving extensions...
Renamed 100 image(s) in: D:\Faisal sir Dataset-20250809T142456Z-1-001\Faisal sir Dataset\digit\0
- DIGIT_0_1.png
- DIGIT_0_2.png
- DIGIT_0_3.png
- DIGIT_0_4.png
- DIGIT_0_5.png
- DIGIT_0_6.png
- DIGIT_0_7.png
- DIGIT_0_8.png
- DIGIT_0_9.png
- DIGIT_0_10.png
...
